In [ ]:
import scipy.io
import numpy as np

healthy =[]
asd = []
hf = scipy.io.loadmat('/content/drive/MyDrive/graph project data/um1/H_comp.mat')
af = scipy.io.loadmat('/content/drive/MyDrive/graph project data/um1/ASD_comp.mat')
hfeatures = hf['H_comp']
afeatures = af['ASD_comp']
for i in range(0, 52):
  amat = scipy.io.loadmat(f'/content/drive/MyDrive/graph project data/um1/a{i+1}.mat')
  hmat = scipy.io.loadmat(f'/content/drive/MyDrive/graph project data/um1/h{i+1}.mat')
  asd.append(np.where(amat['matrix'] > 0.1, 1, 0))
  healthy.append(np.where(hmat['matrix'] > 0.1, 1, 0))
  np.fill_diagonal(healthy[i], 0)
  np.fill_diagonal(asd[i], 0)


In [ ]:
%%capture
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric
!pip install plotly
!pip install --upgrade scipy
!wget "https://gist.githubusercontent.com/mogproject/50668d3ca60188c50e6ef3f5f3ace101/raw/e11d5ac2b83fb03c0e5a9448ee3670b9dfcd5bf9/visualize.py"

In [ ]:
import torch
from torch_geometric.data import Data, InMemoryDataset


In [ ]:
len(asd)

In [ ]:
#1 = asd
#0 = healthy

In [ ]:

data_list = []

for i in range(len(asd)):
  adj_matrix = asd[i]
  adj_tensor = torch.tensor(adj_matrix)
  num_nodes = adj_matrix.shape[0]
  node_indices = torch.arange(num_nodes)
  row_indices, col_indices = torch.nonzero(adj_tensor, as_tuple=True)
  edge_index = torch.stack([row_indices, col_indices], dim=0)
  node_features = afeatures[0,i]
  class_label = 1

    # Create a `Data` object for each graph
  data = Data(x=torch.tensor(node_features), edge_index=edge_index, y=torch.tensor([class_label]))
  data_list.append(data)

  adj_matrix = healthy[i]
  adj_tensor = torch.tensor(adj_matrix)
  num_nodes = adj_matrix.shape[0]
  node_indices = torch.arange(num_nodes)
  row_indices, col_indices = torch.nonzero(adj_tensor, as_tuple=True)
  edge_index = torch.stack([row_indices, col_indices], dim=0)
  node_features = hfeatures[0,i]
  class_label = 0

    # Create a `Data` object for each graph
  data = Data(x=torch.tensor(node_features), edge_index=edge_index, y=torch.tensor([class_label]))
  data_list.append(data)


In [ ]:
import networkx as nx
# General imports
import os
import json
import collections

# Data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import scipy.sparse as sp



# Graph imports
import torch
from torch import Tensor
import torch.nn.functional as F

import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx

import networkx as nx
from networkx.algorithms import community

from tqdm.auto import trange
#External helper packages
%run visualize.py
from visualize import GraphVisualization

In [151]:
from torch_geometric.data import Dataset, Data

class CustomDataset(Dataset):
    def __init__(self, graph_list):
        self.graph_list = graph_list
        super().__init__()

    def len(self):
        return len(self.graph_list)

    def get(self, idx):
      data = self.graph_list[idx]
      data.x = data.x.float()
      return data

from torch_geometric.loader import DataLoader

dataset = CustomDataset(data_list)
#dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [152]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')



Dataset: CustomDataset(104):
Number of graphs: 104
Number of features: 295
Number of classes: 2

Data(x=[201, 295], edge_index=[2, 8924], y=[1])
Number of nodes: 201
Number of edges: 8924
Average node degree: 44.40
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [153]:
def create_graph(graph):
    g = to_networkx(graph)
    pos = nx.spring_layout(g)
    vis = GraphVisualization(
        g, pos, node_text_position='top left', node_size=20,
    )
    fig = vis.create_figure()
    return fig

fig = create_graph(dataset[0])
fig.show()

In [154]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:84]
test_dataset = dataset[84:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 84
Number of test graphs: 20


In [155]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
DataBatch(x=[6432, 295], edge_index=[2, 358810], y=[32], batch=[6432], ptr=[33])

Step 2:
Number of graphs in the current batch: 32
DataBatch(x=[6432, 295], edge_index=[2, 399736], y=[32], batch=[6432], ptr=[33])

Step 3:
Number of graphs in the current batch: 20
DataBatch(x=[4020, 295], edge_index=[2, 216802], y=[20], batch=[4020], ptr=[21])



In [ ]:
"""from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import SAGEConv , GATv2Conv

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        #self.conv1 = SAGEConv(dataset.num_node_features, hidden_channels , aggr = 'lstm')
        #self.conv2 = GATv2Conv(hidden_channels, hidden_channels)
        #self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        #print(x.type)
        x = self.lin(x)

        return x

model = GNN(hidden_channels=64)
print(model)"""

GNN(
  (conv1): GCNConv(295, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [185]:
import torch
import torch.nn as nn
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, hidden_dim)
        self.lin = Linear(hidden_dim, 2)
        #self.sigmoid = nn.Sigmoid()
    def forward(self, data, batch):
        x = self.conv1(data.x, data.edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)

        x = self.conv2(x, data.edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)

        x = self.conv3(x, data.edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5)
        #print(x.type)
        x = self.lin(x)
        #x = self.sigmoid(x)
        return F.log_softmax(x, dim=1)

model = GraphSAGE(in_dim=dataset[0].num_node_features,
                 hidden_dim=16,
                 out_dim=2)
print(model)

GraphSAGE(
  (conv1): SAGEConv(295, 16, aggr=mean)
  (conv2): SAGEConv(16, 16, aggr=mean)
  (conv3): SAGEConv(16, 16, aggr=mean)
  (lin): Linear(in_features=16, out_features=2, bias=True)
)


In [186]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
"""
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data,data.batch)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
return loss.item()
"""
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data, data.batch)  # Perform a single forward pass.
         #out = out.to(data.y.dtype)
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data, data.batch)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 50):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5595, Test Acc: 0.6500
Epoch: 002, Train Acc: 0.7381, Test Acc: 0.5000
Epoch: 003, Train Acc: 0.7500, Test Acc: 0.5500
Epoch: 004, Train Acc: 0.8095, Test Acc: 0.6500
Epoch: 005, Train Acc: 0.8929, Test Acc: 0.5500
Epoch: 006, Train Acc: 0.9524, Test Acc: 0.6500
Epoch: 007, Train Acc: 0.9167, Test Acc: 0.6500
Epoch: 008, Train Acc: 0.9524, Test Acc: 0.7500
Epoch: 009, Train Acc: 0.9762, Test Acc: 0.6000
Epoch: 010, Train Acc: 1.0000, Test Acc: 0.8000
Epoch: 011, Train Acc: 0.9762, Test Acc: 0.6500
Epoch: 012, Train Acc: 0.9881, Test Acc: 0.6500


KeyboardInterrupt: ignored

In [ ]:
"""for data in train_loader:  # Iterate in batches over the training dataset.
  out = model(data, data.batch)  # Perform a single forward pass.
  print(out.type)
  print(out)
  print(data.y.type)
  print(data.y)
  #out = out.to(data.y.dtype)
  loss = criterion(out, data.y)"""